In [2]:
import numpy as np
import pandas as pd

In [3]:
# Load the Berlin grocery data
df = pd.read_csv('3_task_4_no_duplicates.csv', low_memory=False, encoding='utf-8')

In [5]:
df.head(3)

,UID,PRODUCT_NAME,IMAGE_URL,PRODUCT_PRICE,PRODUCT_PRICE_TREATED_OUTLIERS,PRODUCT_LINK,PRODUCT_INFORMATION_T,CATEGORY,SUB_CATEGORY,PRICE_PER_KG/L,...,PRODUCT_REVIEWS,STORE_LINK,STORE_NAME,PRICE_PER_KG/L_UNIT,ENERGY_KJ,SUB_SUB_CATEGORY,PRODUCT_BRAND,PRODUCT_INFORMATION,MANUFACTURER,NUTRITIONAL_LABEL
0,138,Almdudler Original Herb Lemonade,https://imageproxy.wolt.com/menu/menu-images/6...,2.24,2.24,https://wolt.com/en/deu/berlin/venue/flink-kar...,Alpine herbal lemonade,Food & Beverage,Soft Drinks,1.99,...,NaN,NaN,Wolt: Flink Karl Liebknecht,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,151,Almdudler Sugar Free Herb Lemonade,https://imageproxy.wolt.com/menu/menu-images/6...,2.24,2.24,https://wolt.com/en/deu/berlin/venue/flink-kar...,Alpine herb lemonade without sugar with sweete...,Food & Beverage,Soft Drinks,1.99,...,NaN,NaN,Wolt: Flink Karl Liebknecht,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,267,almond butter brown 250g,https://static.mueller.de/markant_041044201797...,5.99,5.99,https://www.mueller.de/p/alnatura-mandelmus-br...,Product information An intensely aromatic pure...,Food & Beverage,Sweet spreads,23.96,...,NaN,NaN,Muller,NaN,2559.002825,NaN,NaN,NaN,ALNATURA,very unhealthy


In [6]:
# Columns taken into consideration for modelling
df_ = df[['PRODUCT_NAME', 'PRODUCT_PRICE_TREATED_OUTLIERS', 'PRODUCT_INFORMATION_T', 
       'CATEGORY', 'SUB_CATEGORY', 'SATURATED_FATTY_ACIDS', 'CARBOHYDRATES', 'SUGAR',
       'PROTEIN', 'FIBER', 'SALT', 'FAT', 'INGREDIENTS', 'SUB_SUB_CATEGORY']].copy()

# Reasons for not taking into consideration the following columns:
# IMAGE_URL: Unless we compare the images themselves, which would be a very complex process in itself, 
#      adding this column would not bring us any benefit
# PRODUCT_PRICE. PRODUCT_PRICE_TREATED_OUTLIERS will be used instead
# PRODUCT_LINK: doesn't add valuable information
# PRODUCT_QUANTITY: Upon inspection, this column information is highly unreliable
# PRICE_PER_KG/L: Because the PRODUCT_QUANTITY column is unreliable, there's a high chance this one is also unreliable
# ALLERGENS: Most of the values are missing
# STORAGE_INFORMATION: Visual testing of the model's results proved that including this column would provide worse results
# STORE_LINK, STORE_NAME: Wanted to use only the intrinsic characteristics of the products to create product similarities
# PRODUCT_BRAND, MANUFACTURER: For most products, the brand is already present in the title, and matching products by brand
#      instead of their characteristics was something we tried to avoid

In [7]:
# Remove generic product information, like packaging information, warnings, or any other type of information
#      that doesn't add specific details about the products and artificially creates similarities between products
generic_text = [
    "nan",
    "KL. II",
    " No detailed information was provided about this product.",
    "Westfalenland Fleischwaren GmbH has provided the above information.",
    "The product design, the commercial class and the packer may differ when the goods are delivered. Please check the information on the respective product packaging and the delivery note, only these are binding.",
    "DE-Ã?KO-006", "DE-Ã?KO-001", "DE-Ã?KO005", "DE-Ã?KO-005", "DE-Ã?KO-037", "DE-Ã?KO-013",
    "Product information ",
    "Die Produktverpackung und zugehörigen Dokumente enthalten möglicherweise Angaben, die über die auf unserer Internetseite gemachten Angaben hinausgehen und/oder sich von ihnen unterscheiden. Wir empfehlen Ihnen daher, sich nicht allein auf die Angaben zu verlassen, die auf unserer Internetseite angezeigt werden, sondern sich vor Gebrauch der Ware stets auch sorgfältig die Etiketten, Warnhinweise und Anleitungen durchzulesen, die mit der Ware geliefert werden.",
    "List of ingredients: ",
    "*",
    "Schwartauer Werke has provided the above information.",
]
for text in generic_text:
    df_['PRODUCT_INFORMATION_T'] = df_['PRODUCT_INFORMATION_T'].str.replace(text, "", regex=False)
    df_['INGREDIENTS'] = df_['INGREDIENTS'].str.replace(text, "", regex=False)

In [8]:
# Convert empty strings to NaN values
df_.replace("", np.nan, inplace=True)

In [9]:
# Because we use text comparison in our model, it is more valuable to transform the numerical columns into categorical text.
# And in this, way we also address the skewness identified in these columns
num_to_cat_cols = {"FAT": df_['FAT'].describe()["75%"], 
                   "SALT": df_['SALT'].describe()["75%"], 
                   "CARBOHYDRATES": df_['CARBOHYDRATES'].describe()["75%"], 
                   "PROTEIN": df_['PROTEIN'].describe()["75%"], 
                   "SUGAR": df_['SUGAR'].describe()["75%"], 
                   "FIBER": df_['FIBER'].describe()["75%"], 
                   "SATURATED_FATTY_ACIDS": df_['SATURATED_FATTY_ACIDS'].describe()["75%"]}

def transform_col(x, col, low_upper_limit):
    if np.isnan(x):
        return 'Unknown ' + col.replace("_", " ")
    if x == 0:
        return 'No ' + col.replace("_", " ")
    if x > 0 and x <= low_upper_limit:
        return 'Low ' + col.replace("_", " ")
    return 'High ' + col.replace("_", " ")

for col, low_upper_limit in num_to_cat_cols.items():
    df_.loc[:,col+'_BINNED'] = df_[col].apply(transform_col, args=(col, low_upper_limit))

df_.drop(['FAT', 'SALT', 'FIBER', 'CARBOHYDRATES', 'PROTEIN', 'SUGAR', 'SATURATED_FATTY_ACIDS'], 
         axis=1, inplace=True)

In [10]:
# Select the final list of text features we'll use to build our model and combine them in a single phrase
text_features = ['PRODUCT_NAME', 'PRODUCT_INFORMATION_T', 'INGREDIENTS', 
       'SUB_CATEGORY', 'SUB_SUB_CATEGORY',
       'FAT_BINNED', 'SALT_BINNED', 'CARBOHYDRATES_BINNED', 'PROTEIN_BINNED', 
       'SUGAR_BINNED', 'FIBER_BINNED', 'SATURATED_FATTY_ACIDS_BINNED']

# text_features = ['PRODUCT_NAME', 'PRODUCT_INFORMATION_T', 'INGREDIENTS', 'SUB_CATEGORY', 'SUB_SUB_CATEGORY']

df_['metadata'] = df_[text_features].apply(lambda x: '. '.join(x.dropna()), axis=1)

# The CATEGORY column was not taken into consideration for the final modeling because it proved to produce worse results
# This may be due to containing generic categories like Other, which would put very different products in the same category,
#      and at the same time, a lot of products were put in the wrong category
# On the contrary, SUB_CATEGORY, SUB_SUB_CATEGORY contain a lot of specific information about the products

### Modeling

In [12]:
# SentenceTransformers is a Python framework for state-of-the-art sentence, text, and image embeddings.
# The all-MiniLM-L6-v2 model maps sentences & paragraphs to a 384-dimensional dense vector space
# More sentence-transforming models can be found at: https://huggingface.co/sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\USER\miniconda3\envs\recsys\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# get embeddings
# TO DO: Split operation into chunks or use other method to keep track of progress, because this is a the lengthy process
sentence_embeddings = model.encode(df_['metadata'].tolist())

In [ ]:
# Save a version without product price
np.save("final_matrix_f64_noP", final_matrix)

In [14]:
# Scale the PRODUCT_PRICE_TREATED_OUTLIERS column and add it to the sentence_embeddings matrix
def my_scaler(min_scale_num,max_scale_num,var):
    return (max_scale_num - min_scale_num) * ( (var - min(var)) / (max(var) - min(var)) ) + min_scale_num

df['PRODUCT_PRICE_TREATED_OUTLIERS_SCALED'] = my_scaler(0, 1, df['PRODUCT_PRICE_TREATED_OUTLIERS'].astype(float)) # scaled between 0,1

final_matrix = np.hstack((sentence_embeddings, df["PRODUCT_PRICE_TREATED_OUTLIERS_SCALED"].to_numpy().reshape(-1,1)))

In [15]:
np.save("final_matrix_f64", final_matrix)

### Cosine Similarity

In [16]:
# Compute similarity
# Split operation into chucks, otherwise it would be impossible to compute the operation because ~50GB of RAM would be required
# For our application, we only need to keep the top 20 most similar products for each product instead of the complete 87157x87157 similarities

from sklearn.metrics.pairwise import cosine_similarity

chunk = 10000
steps = int(final_matrix.shape[0] / chunk)+1
top_k = 21
similarity_top_k = np.empty((final_matrix.shape[0],top_k), dtype="uint32")

for i in range(steps):
    if ((i+1)*chunk) > final_matrix.shape[0]:
        upper_bound = final_matrix.shape[0]
    else:
        upper_bound = (i+1)*chunk

    similarity_matrix = cosine_similarity(final_matrix[i*chunk:upper_bound], final_matrix)

    for j in range(similarity_matrix.shape[0]):
        similarity_top_k[i*chunk+j] = np.argsort(similarity_matrix[j])[::-1][:top_k]

In [18]:
np.save("similarity_top_k", similarity_top_k)

### Testing

In [19]:
# Print the top 20 similar product names
product_id = 3456

for i in similarity_top_k[product_id]:
    print("{}: {} {}".format(i, df.iloc[i]['PRODUCT_PRICE_TREATED_OUTLIERS'], df.iloc[i]['PRODUCT_NAME']))

3456: 12.99 Nescafe Gold Mild 200G
53631: 11.99 Nescafe Gold Mild Instant Coffee 200g
53630: 11.99 Nescafe Gold Instant Coffee 200g
53633: 6.49 Nescafe Gold Original instant coffee 100g
3455: 7.99 Nescafe Gold Espresso 100G
3458: 12.99 Nescafe Gold Original 200G
3457: 7.49 Nescafe Gold Original 100G
3453: 12.99 Nescafe Gold Crema 200G
3446: 8.49 Nescafe Classic 200G
3445: 5.79 Nescafe Classic 100G
53561: 6.32 NESCAFÉ Gold Crema, instant coffee, 200g glass, pack of 3
53492: 6.32 Nescafé Classic instant coffee beans (roasted medium dark), 200g
53512: 2.99 Nescafé Gold Type Latte, Soluble Bean Coffee, Instant Coffee, Instant Coffee, 6 x 8 Servings
53511: 2.99 Nescafé Gold type ESPRESSO, pack of 6 (6 x 100 g)
53572: 24.85 NESCAFÉ Gold Type Cappuccino
3447: 8.49 Nescafe Classic Crema 200G
53603: 13.62 Nescafe Azera Americano Instant Coffee (100g) by Nescafe
53634: 66.21 Nescafe Gold Premium Blend 6 x 200g
53586: 19.08 NESCAFÉ Gold Type Latte Vanilla 4x(8x18.5g)
53570: 12.99 NESCAFÉ GOLD Ori